In [1]:
from kika.input.material import Material, MaterialCollection, Nuclide
from kika.input.parse_materials import read_material
import numpy as np

In [2]:
# Create a simple water material with atomic fractions
water = Material(id=1, name="Water")

# Add nuclides: H-1 and O-16
water.add_nuclide(nuclide=1001, fraction=2, fraction_type='ao')
water.add_nuclide(nuclide=8016, fraction=1, fraction_type='ao')

In [3]:
water

              MCNP Material (ID: 1)               

Total Nuclides: 2

Fraction Type: Atomic

--------------------------------------------------
  ZAID   | Element  |    Fraction   
--------------------------------------------------
  1001   |    H     |  2.000000e+00 
  8016   |    O     |  1.000000e+00 
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .expand_natural_elements() - Expand natural elements to isotopes
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nuclide['Fe56'] - Access nuclide by symbol
- .nuclide[26056] - Access nuclide by ZAID
- .nuclide['Fe'] or .nuclide[26000] - Access natural Fe
- print(material) - Print material in MCNP format

In [4]:
steel = Material(
    id=2,
    name="Stainless Steel 316",
    libs={'nlib': '80c', 'plib': '12p'}
)

steel.add_nuclide(26056, 0.68, 'ao', library='70c')  # Fe-56
steel.add_nuclide(24052, 0.17, 'ao')  # Cr-52
steel.add_nuclide(28058, 0.12, 'ao')  # Ni-58
steel.add_nuclide(42095, 0.02, 'ao')  # Mo-95
steel.add_nuclide(25055, 0.01, 'ao')  # Mn-55
steel.add_element('C', 0.002, 'ao')   # Carbon

In [5]:
steel

              MCNP Material (ID: 2)               

MCNP Libraries: nlib=80c, plib=12p
Total Nuclides: 6

Fraction Type: Atomic

--------------------------------------------------
  ZAID   | Element  |    Fraction    |  Libraries  
--------------------------------------------------
  6000   |    C     |  2.000000e-03  |      -      
 24052   |    Cr    |  1.700000e-01  |      -      
 25055   |    Mn    |  1.000000e-02  |      -      
 26056   |    Fe    |  6.800000e-01  |     70c     
 28058   |    Ni    |  1.200000e-01  |      -      
 42095   |    Mo    |  2.000000e-02  |      -      
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .expand_natural_elements() - Expand natural elements to isotopes
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nuclide['Fe56'] - Access nuclide by symbol
- .nuclide[

In [6]:
steel.expand_natural_elements()

              MCNP Material (ID: 2)               

MCNP Libraries: nlib=80c, plib=12p
Total Nuclides: 7

Fraction Type: Atomic

--------------------------------------------------
  ZAID   | Element  |    Fraction    |  Libraries  
--------------------------------------------------
  6012   |    C     |  1.978600e-03  |      -      
  6013   |    C     |  2.140000e-05  |      -      
 24052   |    Cr    |  1.700000e-01  |      -      
 25055   |    Mn    |  1.000000e-02  |      -      
 26056   |    Fe    |  6.800000e-01  |     70c     
 28058   |    Ni    |  1.200000e-01  |      -      
 42095   |    Mo    |  2.000000e-02  |      -      
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .expand_natural_elements() - Expand natural elements to isotopes
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nu

In [7]:
steel.nuclide['Fe56']

                            Nuclide: Fe56                             
ZAID:                     26056
Element:                  Fe
Mass Number:              56
Fraction:                 6.800000e-01
Is Natural:               False

Library Overrides:
  nlib                    70c

## Test MCNP Format with Comments

Testing the new MCNP output format that includes:
1. Material name as comment before material card
2. Density in g/cc and atoms/b-cm as inline comment
3. Nuclide symbol as inline comment after each fraction

In [8]:
# Create a material with name and density
uo2 = Material(id=10, name="Uranium Dioxide (UO2)")
uo2.add_nuclide('U235', 0.03, 'ao')
uo2.add_nuclide('U238', 0.97, 'ao')
uo2.add_nuclide('O16', 2.0, 'ao')
uo2.set_density(10.97, 'g/cc')  # Set density

# Normalize to atomic fractions
uo2.normalize('ao')

print("=== Material with name and density ===")
print(uo2.to_mcnp())

=== Material with name and density ===
c KIKA_MAT_NAME: Uranium Dioxide (UO2)
m10  $ KIKA_DENSITY: 1.097000e+01 g/cc, 7.343045e-02 atoms/b-cm
     8016           6.666667e-01   $ O16   
     92235          1.000000e-02   $ U235  
     92238          3.233333e-01   $ U238  


In [9]:
# Test round-trip: write to MCNP format and parse it back
mcnp_output = uo2.to_mcnp()
lines = mcnp_output.split('\n')

print("=== Parsing the MCNP output ===")
print(f"Lines to parse: {lines}")
print()

# Find the material line (starts with 'm')
mat_start_idx = None
for i, line in enumerate(lines):
    if line.strip().lower().startswith('m') and not line.strip().lower().startswith('mt'):
        import re
        if re.match(r'^m\d+', line.strip().lower()):
            mat_start_idx = i
            break

print(f"Material starts at line index: {mat_start_idx}")

# Parse the material
parsed_material, end_idx = read_material(lines, mat_start_idx)

print(f"\n=== Parsed Material ===")
print(f"ID: {parsed_material.id}")
print(f"Name: {parsed_material.name}")
print(f"Density: {parsed_material.density} {parsed_material.density_unit}")
print(f"Fraction type: {parsed_material.fraction_type}")
print(f"Nuclides: {list(parsed_material.nuclide.keys())}")

=== Parsing the MCNP output ===
Lines to parse: ['c KIKA_MAT_NAME: Uranium Dioxide (UO2)', 'm10  $ KIKA_DENSITY: 1.097000e+01 g/cc, 7.343045e-02 atoms/b-cm', '     8016           6.666667e-01   $ O16   ', '     92235          1.000000e-02   $ U235  ', '     92238          3.233333e-01   $ U238  ']

Material starts at line index: 1

=== Parsed Material ===
ID: 10
Name: Uranium Dioxide (UO2)
Density: 10.97 g/cc
Fraction type: ao
Nuclides: ['O16', 'U235', 'U238']


In [10]:
# Test a material without name or density
simple_mat = Material(id=5)
simple_mat.add_nuclide('Fe56', 0.92, 'ao', library='80c')
simple_mat.add_nuclide('C12', 0.08, 'ao', library='80c')

print("=== Material without name/density ===")
print(simple_mat.to_mcnp())

# Also test parsing this format
lines2 = simple_mat.to_mcnp().split('\n')
parsed2, _ = read_material(lines2, 0)
print(f"\nParsed - ID: {parsed2.id}, Name: {parsed2.name}, Density: {parsed2.density}")

=== Material without name/density ===
m5
     6012.80c       8.000000e-02   $ C12   
     26056.80c      9.200000e-01   $ FE56  

Parsed - ID: 5, Name: None, Density: None


In [11]:
# Test parsing a traditional MCNP format (without KIKA comments)
traditional_mcnp = """c Some random comment
c Another comment  
m1 nlib=80c
     1001  0.666667  $ hydrogen
     8016  0.333333  $ oxygen
"""

lines3 = traditional_mcnp.strip().split('\n')

# Find material start
for i, l in enumerate(lines3):
    if l.strip().startswith('m1'):
        mat_start = i
        break

parsed3, _ = read_material(lines3, mat_start)
print("=== Parsing traditional MCNP format ===")
print(f"ID: {parsed3.id}")
print(f"Name: {parsed3.name}")  # Should be None (not a KIKA comment)
print(f"Density: {parsed3.density}")  # Should be None
print(f"Libs: {parsed3.libs}")
print(f"Nuclides: {list(parsed3.nuclide.keys())}")

=== Parsing traditional MCNP format ===
ID: 1
Name: None
Density: None
Libs: {'nlib': '80c'}
Nuclides: ['H1', 'O16']


In [12]:
# Test weight fraction material with density
concrete = Material(id=100, name="Concrete")
concrete.add_nuclide('H1', 0.01, 'wo')
concrete.add_nuclide('O16', 0.532, 'wo')
concrete.add_nuclide('Si28', 0.337, 'wo')
concrete.add_nuclide('Ca40', 0.044, 'wo')
concrete.add_nuclide('Fe56', 0.014, 'wo')
concrete.add_nuclide('Al27', 0.034, 'wo')
concrete.set_density(2.3, 'g/cc')

print("=== Concrete (weight fractions) ===")
print(concrete.to_mcnp())

# Verify round-trip
lines4 = concrete.to_mcnp().split('\n')
for i, l in enumerate(lines4):
    if l.strip().lower().startswith('m100'):
        mat_start = i
        break

parsed4, _ = read_material(lines4, mat_start)
print(f"\n=== Round-trip verification ===")
print(f"Name: {parsed4.name}")
print(f"Density: {parsed4.density} {parsed4.density_unit}")
print(f"Fraction type: {parsed4.fraction_type}")

=== Concrete (weight fractions) ===
c KIKA_MAT_NAME: Concrete
m100  $ KIKA_DENSITY: 2.300000e+00 g/cc, 8.250644e-02 atoms/b-cm
     13027         -3.400000e-02   $ AL27  
     20040         -4.400000e-02   $ CA40  
     26056         -1.400000e-02   $ FE56  
     1001          -1.000000e-02   $ H1    
     8016          -5.320000e-01   $ O16   
     14028         -3.370000e-01   $ SI28  

=== Round-trip verification ===
Name: Concrete
Density: 2.3 g/cc
Fraction type: wo


In [13]:
# Reload module to get the updated code
import importlib
import kika.input.material
importlib.reload(kika.input.material)
from kika.input.material import Material

# Test aligned columns
test_mat = Material(id=99, name="Test Alignment")
test_mat.add_nuclide('H1', 0.1, 'ao', '80c')       # Short ZAID: 1001
test_mat.add_nuclide('O16', 0.2, 'ao', '80c')      # Short ZAID: 8016  
test_mat.add_nuclide('Fe56', 0.3, 'ao', '70c')     # Medium ZAID: 26056
test_mat.add_nuclide('U235', 0.2, 'ao', '80c')     # Long ZAID: 92235
test_mat.add_nuclide('Pu239', 0.2, 'ao', '80c')    # Long ZAID: 94239
test_mat.set_density(5.0, 'g/cc')

print("=== Aligned columns test ===")
print(test_mat.to_mcnp())

=== Aligned columns test ===
c KIKA_MAT_NAME: Test Alignment
m99  $ KIKA_DENSITY: 5.000000e+00 g/cc, 2.620813e-02 atoms/b-cm
     26056.70c      3.000000e-01   $ FE56  
     1001.80c       1.000000e-01   $ H1    
     8016.80c       2.000000e-01   $ O16   
     94239.80c      2.000000e-01   $ PU239 
     92235.80c      2.000000e-01   $ U235  
